In [1]:
import pandas as pd
import os
import json
import numpy as np

In [2]:
filtered_abs_path = '2021_filtered_data' # We do not upload this data because the information is raw from SafeGraph
dfs = os.listdir(filtered_abs_path)
dfs = [filtered_abs_path + '/' + i for i in dfs if '.csv' in i]
data = [pd.read_csv(i) for i in dfs]
poi = pd.concat(data)
poi = poi.drop_duplicates()
poi['week'] = poi['date_range_start'].apply(lambda x: x[:10])

In [4]:
unique_weeks = list(poi['week'].unique())
table_2018 = pd.read_csv('data/2_data_for_PSO_calibration/table_2018.csv')

In [5]:
table_2018.head()

,Unnamed: 0,A_cbg,B_store,C_Percentage_of_Visits_2018,D_Number_of_Visits_2018,E_Population_in_this_cbg,F_Population_where_store_is,G_Distance_between_cbg_and_store,H_Area_of_store,I_Includes_parking_lot,J_POI_count_where_store_is,K_POI_diversity_where_store_is,L_Demographic_similarity,M_Median_Income_in_this_cbg,N_Median_Income_where_store_is,O_Brand_name,P_safegraph_brand_id,Q_Number_of_Visits_by_brand_2018,R_Percentage_of_Visits_by_brand_2018
0,0,360810140001,sg:51c8d943ff72400ab86e7e8896f4b5af,0.000000,0,2187,287,9.780264,135702,0.0,72,3.083034,1.000000,69750.0,36006.0,Target,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e,24,0.089552
1,1,360810140001,sg:f189f91419974d03a20257132d73da58,0.044776,12,2187,1348,4.111893,149385,1.0,69,2.781085,1.000000,69750.0,80663.0,Target,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e,24,0.089552
2,2,360810140001,sg:c7222887b3834a58a6bc40c168cdb38a,0.000000,0,2187,1280,8.292913,3678,0.0,6,1.560710,0.000379,69750.0,0.0,Target,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e,24,0.089552
3,3,360810140001,sg:e2543bebe82742a1941c21660d9f4168,0.000000,0,2187,1796,8.162305,30554,0.0,39,2.724921,1.000000,69750.0,75547.0,Target,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e,24,0.089552
4,4,360810140001,sg:3275834c58d241baacdb9965f5003af6,0.000000,0,2187,163,8.393494,6135,1.0,39,2.929817,1.000000,69750.0,0.0,Target,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e,24,0.089552


In [6]:
cbgs = list(table_2018['A_cbg'].unique())
cbgs = [str(c) for c in cbgs]

In [7]:
def extract_max(data, stores_list):
    stores_actual_list = [i for i in stores_list if i in data.columns]
    temp = data[stores_actual_list]
    return temp.max(axis = 1)

In [8]:
list_517E = ['sg:b92ac1661ff14e78b25a99515d2370c5','sg:507b345e2e2445f5b4edaf6fa1b01b0d', 'sg:fcf8054ca0eb4daa8fa77bd2ea68987f']
list_4024_College = ['sg:37cd539e6ba84eb898b7f979a38e08cc','sg:da1cac5e846a4c39ab873c53a240da32', 'sg:83ad2c11ae2f4810a3aa10155a1d7e9e']
list_40W = ['sg:3ee2fa14ed7841c88080d239f4aa7292', 'sg:a5570cc64afd4f12b4bc7a8b68774922']
list_all_original = list_517E + list_4024_College + list_40W

In [9]:
stores_2018 = table_2018['B_store'].unique().tolist()

In [10]:
poi.columns

Index(['placekey', 'safegraph_place_id', 'parent_placekey',
       'parent_safegraph_place_id', 'location_name', 'street_address', 'city',
       'region', 'postal_code', 'iso_country_code', 'safegraph_brand_ids',
       'brands', 'date_range_start', 'date_range_end', 'raw_visit_counts',
       'raw_visitor_counts', 'visits_by_day', 'visits_by_each_hour', 'poi_cbg',
       'visitor_home_cbgs', 'visitor_daytime_cbgs',
       'visitor_country_of_origin', 'distance_from_home', 'median_dwell',
       'bucketed_dwell_times', 'related_same_day_brand',
       'related_same_week_brand', 'device_type', 'visitor_home_aggregation',
       'normalized_visits_by_state_scaling',
       'normalized_visits_by_total_visits',
       'normalized_visits_by_total_visitors',
       'normalized_visits_by_region_naics_visits',
       'normalized_visits_by_region_naics_visitors', 'week'],
      dtype='object')

In [13]:
#sorted(poi['week'].unique())

In [14]:
def get_matrix(region, week):
    df_temp = poi[(poi['week'] == week) & (poi['region'] == region)]
    matrix_info = {}
    
    stores_list = list(poi[poi['region'] == region]['safegraph_place_id'].unique())
    cbg_info = df_temp.visitor_home_cbgs.tolist()
    cbg_info = [json.loads(i) for i in cbg_info] 
    length = len(df_temp)
    
    for store in stores_list:
        matrix_info[store] = {}
        for cbg in cbgs:
            matrix_info[store][cbg] = 0
    
    for i in range(length):
        for k,v in cbg_info[i].items():
            if k in cbgs:
                matrix_info[stores_list[i]][k] += v
    
    ans_df = pd.DataFrame.from_dict(matrix_info)

    ans_df['aggregated_517E'] = extract_max(ans_df, list_517E)
    ans_df['aggregated_4024_College'] = extract_max(ans_df, list_4024_College)
    ans_df['aggregated_40W'] = extract_max(ans_df, list_40W)
    list_all = [i for i in list_all_original if i in ans_df.columns]
    ans_df = ans_df.drop(list_all, axis=1)
    for s_2018 in stores_2018:
        if s_2018 not in ans_df.columns:
            ans_df[s_2018] = [0] * len(ans_df)
    stores_2020_here = list(ans_df.columns)
    for s_2020 in stores_2020_here:
        if s_2020 not in stores_2018:
            ans_df = ans_df.drop(s_2020, axis=1)
    return ans_df
    '''
    A = np.zeros((len(stores_list),len(cbgs)),dtype=int)
    df_cbg = pd.DataFrame(A, index=stores_list, columns=cbgs)
    for k,v in matrix_info.items():
        for k_sub,v_sub in v.items():
            df_cbg.loc[k,k_sub] += v_sub
        print(k + 'ok')
    
    return df_cbg.T
    '''

In [15]:
poi.columns

Index(['placekey', 'safegraph_place_id', 'parent_placekey',
       'parent_safegraph_place_id', 'location_name', 'street_address', 'city',
       'region', 'postal_code', 'iso_country_code', 'safegraph_brand_ids',
       'brands', 'date_range_start', 'date_range_end', 'raw_visit_counts',
       'raw_visitor_counts', 'visits_by_day', 'visits_by_each_hour', 'poi_cbg',
       'visitor_home_cbgs', 'visitor_daytime_cbgs',
       'visitor_country_of_origin', 'distance_from_home', 'median_dwell',
       'bucketed_dwell_times', 'related_same_day_brand',
       'related_same_week_brand', 'device_type', 'visitor_home_aggregation',
       'normalized_visits_by_state_scaling',
       'normalized_visits_by_total_visits',
       'normalized_visits_by_total_visitors',
       'normalized_visits_by_region_naics_visits',
       'normalized_visits_by_region_naics_visitors', 'week'],
      dtype='object')

In [16]:
def save_csv(week_here):
    temp_df = get_matrix('NY', week_here)
    save_path = 'weekly_foot_traffic_matrix_nyc_2021/'+week_here+'.csv' # We do not upload this data because the information is raw from SafeGraph
    temp_df.to_csv(save_path)

In [17]:
for indx, w in enumerate(unique_weeks):
    save_csv(week_here = w)
    print(str(indx) +' finished')

0 finished
1 finished
2 finished
3 finished
4 finished
5 finished
6 finished
7 finished
8 finished
9 finished
10 finished
11 finished
12 finished
13 finished
14 finished
15 finished
16 finished
17 finished
18 finished
19 finished
20 finished
21 finished
22 finished
23 finished
24 finished
25 finished
26 finished
27 finished
28 finished
29 finished
30 finished
31 finished
32 finished
33 finished
34 finished
35 finished
36 finished
37 finished
38 finished
39 finished
40 finished
41 finished
42 finished
43 finished
44 finished
45 finished
46 finished
47 finished
48 finished
49 finished
50 finished
51 finished
